# Initialisation

In [1]:
import os
import pandas as pd
# os.chdir("path_toward_the_folder_containing_the_file_"functions")   # To use if needed
import functions as functions
import pickle
import numpy as np
from openpyxl import load_workbook
pd.set_option("display.max_rows",None,"display.max_columns",None)     # To display the dataframes entirely

# Find the data

In [2]:
# Files

## Source files
data_source='source_files/user_file.xlsx'
data_impacts_source='source_files/impact_file.xlsx'

## Working files

data_impacts_working_file='working_files\impacts.xlsx'
general_working_file='working_files\data.xlsx'
temporary_saving_file=r'working_files\temporary_save_factors.xlsx'

## Results

pickle_file=r'results\pickle'

# Matrices

##Here put the information necessary for the code to be able to identify your matrices

## A matrix
A_1st_column='name of A 1st column (foreground process 1)'
A_last_column='name of A last column (background process 2)'
A_separation_column="name of A last column being a foreground process (foreground process 5)"
A_first_line='name of A first line'
A_last_line='name of A last line'

## Demand matrix

demand_first_line=A_first_line
demand_last_line=A_last_line
demand_first_column='Demand column name'
demand_last_column=demand_first_column

## Bound matrix

bounds_first_line="Name of the line containing the values of the lower boundaries"
bounds_last_line="Name of the line containing the values of the upper boundaries"
bounds_first_column=A_1st_column
bounds_last_column=A_last_column

## Phases matrix
phases_first_line='Name of the first line containing the phases'
phases_last_line='Name of the first line containing the phases'

## Impact matrix

Impacts_first_column=A_1st_column
Impacts_last_column=A_last_column
Impacts_separation_column=A_separation_column
Impacts_IWP_Mid_first_line="IWP Mid | Climate change, long term (kg CO2 eq (long))"
Impacts_IWP_Mid_last_line="IWP Mid | Water scarcity (m3 world-eq)"
Impacts_IWP_End_first_line="IWP End | Ecosystem quality | Climate change, ecosystem quality, long term (PDF.m2.yr)"
Impacts_IWP_End_last_line="IWP End | Human health | Human health total red (DALY)"
Impact_EF_3_0_first_line="EF 3.0 |acidification | accumulated exceedance (ae) (mol H+ eq)"
Impact_EF_3_0_last_line="EF 3.0 | water use, user deprivation potential (deprivation-weighted water consumption) (m3 depriv.)"

# Build the data file

In [3]:
# Searching for the unit impacts of the aggregated processes

#Precise the database you want to use in the variable "database"
       
functions.search_impacts(data_source,
                            data_impacts_source,
                            data_impacts_working_file,
                            Impacts_first_column,
                            Impacts_separation_column,
                            Impacts_last_column,
                            Impacts_IWP_Mid_first_line,
                            Impacts_IWP_Mid_last_line,
                            Impacts_IWP_End_first_line,
                            Impacts_IWP_End_last_line,
                            Impact_EF_3_0_first_line,
                            Impact_EF_3_0_last_line, 
                            database="database 1",         # change to "database 2" if you want to use database 2
                            open_file="n")                 # change to "y" if you want the constructed file to be opened


 Fermer le fichier 

D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\impacts.xlsx

 s'il est ouvert. 
 La procédure prend quelques secondes.


In [6]:
# Final data file construction

data=functions.data_initialisation(data_source,
                                      data_impacts_working_file,
                                      general_working_file,
                                      A_1st_column,
                                      A_last_column,
                                      A_first_line,
                                      A_last_line,
                                      demand_first_line,
                                      demand_last_line,
                                      demand_first_column,
                                      demand_last_column,
                                      bounds_first_line,
                                      bounds_last_line,
                                      bounds_first_column,
                                      bounds_last_column, 
                                      phases_first_line,
                                      phases_last_line,
                                      open_file="n")               # change to "y" if you want the constructed file to be opened


Lancer cette fonction après avoir formaté les données d'impacts dans 
D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\impacts.xlsx

Fermer le fichier 
D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\data.xlsx
s'il est ouvert.



# MOO preparing

This section allows to make caculation with the impact categories to add, transform, remove... categories from the data file, before optimization

In [7]:
# Read objective functions you will put into the final sum
## The objective function has to be selected in the main file by "X" in the column "Selected scenarios". Be careful it is a big X, little x will not work.
list_objective_functions=functions.objective_choice(data_source)
list_objective_functions

['IWP Mid | Climate change, short term (kg CO2 eq (short))',
 'IWP Mid | Fossil and nuclear energy use (MJ deprived)',
 'IWP End | Ecosystem quality | Ecosystem quality total red (PDF.m2.yr)',
 'IWP End | Human health | Human health total red (DALY)']

## Recovery of the desired impact matrix

In [8]:
# To obtain the data necessary to normalize

norm_fact_df=functions.optimisation_to_norm(data,list_objective_functions)
norm_fact_df

IWP Mid | Climate change, short term (kg CO2 eq (short)) has been optimized successfully
IWP Mid | Fossil and nuclear energy use (MJ deprived) has been optimized successfully
IWP End | Ecosystem quality | Ecosystem quality total red (PDF.m2.yr) has been optimized successfully
IWP End | Human health | Human health total red (DALY) has been optimized successfully
Normalization factors calculated !


,ideal_f,nadir_f
"IWP Mid | Climate change, short term (kg CO2 eq (short))",71015773368.305328,71015773368.305328
IWP Mid | Fossil and nuclear energy use (MJ deprived),1073627559581.519043,1073627559581.519043
IWP End | Ecosystem quality | Ecosystem quality total red (PDF.m2.yr),21831792146.928905,21831792146.928905
IWP End | Human health | Human health total red (DALY),64276.267752,64276.267752


<span style='color:red'> The next step will not work if the ideal_f is equal to the nadir_f </span>

## Impact normalisation and weighting

For the multi objective optimization, I transform distinct objective functions into a weighted normalized sum, which is a new unique objective. The normalization allows to give te desired importance to the objective in the optimization process. By weight variation, we can explore the solution space. We work before the optimization process directly on the unit impacts because it is the only parameter the optimizer accepts. The results needs to be "de-normalized" to mean something.

In [9]:
# Recovery of the objective which will be changed

list_obj=[]


for i in range (len(data.sheet_names[3:])):
    impact_matrix=pd.read_excel(data, data.sheet_names[i+3], index_col=0)
    for j in range (len(impact_matrix.index)):
        for k in range (len(list_objective_functions)):
            if impact_matrix.index[j]==list_objective_functions[k]:
                list_obj.append(impact_matrix.iloc[j,:])

# Create the df which will contain the new objective created


final_df=pd.DataFrame(columns=impact_matrix.columns)

#Normalisation : on divise chaque impact unitaire par f_nadir-f_ideal

for i in range (len(list_obj)):
    div_val=norm_fact_df.iloc[i,1]-norm_fact_df.iloc[i,0]
    for j in range (len(list_obj[i])):
        list_obj[i][j]=list_obj[i][j]/div_val


#Weighting : put the weights equals to the importance we want the objective to have compared to others

# Obtain all the weight factors possible combinaisons

L=[0, 0.25, 0.5, 0.75, 1]

weight_factors_list=functions.gen_weights_combinaisons(L, len(list_obj))

# Ajouter ligne par ligne les impacts voulus
for i in range (len (weight_factors_list)):
    index=[str(weight_factors_list[i])]
    values=[]
    for j in range (final_df.shape[1]):
        val=0
        for k in range(len(weight_factors_list[i])):
            val=val+weight_factors_list[i][k]*list_obj[k].iloc[j]
        values.append(val)
    new_column=pd.DataFrame(data=np.array(values), index=final_df.columns.tolist(), columns=index)
    new_line=new_column.transpose()   
    final_df=pd.concat([final_df, new_line], axis=0)
    
# To be able to see little numbers    
pd.set_option('display.float_format', '{:.2E}'.format)
final_df



C:\Users\Lisa\AppData\Local\Temp\ipykernel_7836\801430228.py:23: RuntimeWarning: divide by zero encountered in double_scalars
  list_obj[i][j]=list_obj[i][j]/div_val
C:\Users\Lisa\AppData\Local\Temp\ipykernel_7836\801430228.py:23: RuntimeWarning: invalid value encountered in double_scalars
  list_obj[i][j]=list_obj[i][j]/div_val
C:\Users\Lisa\AppData\Local\Temp\ipykernel_7836\801430228.py:42: RuntimeWarning: invalid value encountered in double_scalars
  val=val+weight_factors_list[i][k]*list_obj[k].iloc[j]
C:\Users\Lisa\AppData\Local\Temp\ipykernel_7836\801430228.py:42: RuntimeWarning: invalid value encountered in double_scalars
  val=val+weight_factors_list[i][k]*list_obj[k].iloc[j]
C:\Users\Lisa\AppData\Local\Temp\ipykernel_7836\801430228.py:42: RuntimeWarning: invalid value encountered in double_scalars
  val=val+weight_factors_list[i][k]*list_obj[k].iloc[j]
C:\Users\Lisa\AppData\Local\Temp\ipykernel_7836\801430228.py:42: RuntimeWarning: invalid value encountered in double_scalars
 

,P0 : Landfill,P1 : Synthetic turf,P2 : Molded objects,P3 : Recycling in production,P4a : Steel works,P4b : Steel works,P5 : Cement kilns,P6 : Pyrolysis,P7 : Urban heating,P8 : Waste to energy,P9 : Retaining structures,P10 : Civil engineering infill,P11 : Thin civil engineering infill,Non collected tire,Material loss,P6a : Pyrolysis char to carbon black,P6b : Pyrolysis oil to diesel,P4c : Steel works after pyrolysis,Zero impact end-of-life,PG : Retreading,PA : Production,PB1 : Normal use,PB2 : Use of retreaded tires,PC1a : Collect/sorting,PC1b : Collect/sorting,PC2 : Collect/sorting,PD : Mechanical shredding,PE : Mechanical granulation,PF : Cryogenic pulverization,Cement works whole tire conversion,Cement works shredded tire conversion,Synthetic rubber production (SBR),Natural rubber production,Carbon black production,Steel wire production,Silica production,Polyester fibre production,Nylon production,Plasticizer production,Cobalt salt production,Synthetic rubber avoided production (SBR),Natural rubber avoided production,Carbon black avoided production,P1' : Avoided standard synthetic turf production,P2' : Avoided standard molded objects production,P4a' : Avoided standard steel production,P4b' : Avoided standard ELT steel scrap in steel production,P4c' : Avoided standard ELT steel scrap from pyrolysis in steel production,P5' : Avoided standard cement production,P7' : Avoided standard urban heating,P9' : Avoided standard retaining structure construction,P10' : Avoided standard civil engineering fill production,P11' : Avoided standard thin civil engineering infill production,"market group for heat, district or industrial, natural gas and other than natural gas | heat, district or industrial, natural gas and other than natural gas | Cutoff, S - RER","market group for heat, district or industrial, natural gas and other than natural gas | heat, district or industrial, natural gas and other than natural gas | Cutoff, S - RER avoided",d1488aff-796a-4754-bec5-df748fe26fe7_27da8130-82ba-485c-a800-b89efdcb0491,66756d1d-52a8-4aa4-bcbd-f47a8a510907_6047c525-49b7-4a7d-baf6-a61283e5eda9,678f119e-7567-45a9-b4fe-ed83077c6c75_8cb650d5-8a48-4719-81b1-2080b3b4387f,03896bd8-486a-4014-8939-0e4be626d87d_cb262c3c-a7c7-4781-82e6-e1e90dd92b65,9ef4800b-78d1-42ec-99c1-83e572bd0e50_c5adb1fb-872e-4446-a3bb-c4b61aa4bd45,6b3d16db-b5fd-4018-add0-b23f075c4a72_06b8d05b-e1e6-427b-929c-bb55e84e7c08,ac0ff6ad-f9c3-40dd-8f9a-640527708179_99ba8208-7a4c-46e7-b4f0-6d93a8332c0b,3061354a-c549-4e7b-b8a0-b809b877d512_bd20be8e-9b7a-4391-980f-4ecf8f2867be,6b196b75-14ca-45a8-8a3c-ef21c9623784_759b89bd-3aa6-42ad-b767-5bb9ef5d331d,6b196b75-14ca-45a8-8a3c-ef21c9623784_759b89bd-3aa6-42ad-b767-5bb9ef5d331d.1,64677e54-f85a-4a13-bcf2-5532578c801c_159edf36-233e-4e2c-aec0-b0ac0d49f528,cb579c5c-50e8-461b-a629-b77d1a8b5152_76ab0018-e176-43e0-b42c-12e82b937fef,f7e4c223-1cd4-42e0-85d0-98e25ae104ff_e180c07f-fa94-4f7c-8a2f-32cb78f52bd6,17b027cf-474d-4c46-953b-c9d51d0f6fee_291fc06d-1b3e-4077-aabb-346b588ed24b,17b027cf-474d-4c46-953b-c9d51d0f6fee_291fc06d-1b3e-4077-aabb-346b588ed24b.1,0d339822-8369-47bb-bc53-f36a12836018_895b9704-1eef-4395-a880-cb5ca0001bba,193fc1cf-28ea-4afc-a2a5-4fa6c5675d84_0da5307e-df93-4bc2-b5bc-6cda3cb55e8c,0f8ceb9c-2e08-4414-a4f7-07599e4b371f_0d3eda5a-4601-4573-9549-0701c459ab88,2bb489cd-7ac3-44bd-88aa-2b0f0797d05f_1456f165-52a4-47f3-a5c3-e13dfb13626d,c72cb8cf-a36f-4bca-afde-aa76c12f0e18_a2fd8c95-f2db-436a-9f24-f33b17f7975b,ec0d0ef5-0127-433b-9558-793ac300935a_2966d161-5281-47d6-86d3-efbb0a4b2110,d58fe8e0-f2bf-4665-9250-b05cd82f45e3_eb6b5a9a-c482-4c75-9ade-bbf8d89234fe,1a98da5d-7c1c-4973-a1c9-84bafa48b616_d4ee8f39-342b-4443-bbb9-c49b6801b5d6,6fc56491-ad30-47d4-b742-08bb893d9485_37eceabd-b33f-4757-a4b9-5c51dee1710d,3d1de8e2-71d5-4759-9af8-c273736d2f67_a9007f10-7e39-4d50-8f4a-d6d03ce3d673,cfb23737-286f-44c9-aa52-d4439250c239_249c83b0-ff03-45aa-9a5c-f7a58d0ff6e6,9761d70f-cacb-439a-be35-cbfe4d12c231_f51d7ccf-0bee-430d-98a3-8334adbe39fc
"(1, 0, 0, 0)",NaN,NaN,NaN,NaN,NaN,Na

In [8]:
# Add the new objective functions to the main working file

with pd.ExcelWriter(general_working_file, engine='openpyxl', mode='a') as writer:  
    final_df.to_excel(writer, sheet_name='Multi_obj_optim')

# Multi-objective optimisation

Provide scenarios optimized for each selected objective function, which can be linear combination of others (weighted sums)

In [9]:
# If you already have your data file

data=pd.ExcelFile(r"D:\Matières\Double_diplôme_Montreal\CIRAIG\Projet_de_maîtrise\Code\Working_files\data.xlsx")

## Optimisation

In [10]:
# Read objective functions
## The objective function has to be selected in the main file by "X" in the column "Selected scenarios". Be careful it is a big X, little x will not work.
list_objective_functions=functions.objective_choice(data_source)
list_objective_functions

['IWP Mid | Climate change, short term (kg CO2 eq (short))',
 'IWP Mid | Fossil and nuclear energy use (MJ deprived)',
 'IWP End | Ecosystem quality | Ecosystem quality total red (PDF.m2.yr)',
 'IWP End | Human health | Human health total red (DALY)']

In [11]:
# Add the new objective functions to the analysis

additional_objective_functions=pd.read_excel(general_working_file, 'Multi_obj_optim', index_col=0).index.tolist()
list_objective_functions_final=list_objective_functions+additional_objective_functions
list_objective_functions_final

['IWP Mid | Climate change, short term (kg CO2 eq (short))',
 'IWP Mid | Fossil and nuclear energy use (MJ deprived)',
 'IWP End | Ecosystem quality | Ecosystem quality total red (PDF.m2.yr)',
 'IWP End | Human health | Human health total red (DALY)',
 '(1, 0, 0, 0)',
 '(0.75, 0.25, 0, 0)',
 '(0.75, 0, 0.25, 0)',
 '(0.75, 0, 0, 0.25)',
 '(0.5, 0.5, 0, 0)',
 '(0.5, 0.25, 0.25, 0)',
 '(0.5, 0.25, 0, 0.25)',
 '(0.5, 0, 0.5, 0)',
 '(0.5, 0, 0.25, 0.25)',
 '(0.5, 0, 0, 0.5)',
 '(0.25, 0.75, 0, 0)',
 '(0.25, 0.5, 0.25, 0)',
 '(0.25, 0.5, 0, 0.25)',
 '(0.25, 0.25, 0.5, 0)',
 '(0.25, 0.25, 0.25, 0.25)',
 '(0.25, 0.25, 0, 0.5)',
 '(0.25, 0, 0.75, 0)',
 '(0.25, 0, 0.5, 0.25)',
 '(0.25, 0, 0.25, 0.5)',
 '(0.25, 0, 0, 0.75)',
 '(0, 1, 0, 0)',
 '(0, 0.75, 0.25, 0)',
 '(0, 0.75, 0, 0.25)',
 '(0, 0.5, 0.5, 0)',
 '(0, 0.5, 0.25, 0.25)',
 '(0, 0.5, 0, 0.5)',
 '(0, 0.25, 0.75, 0)',
 '(0, 0.25, 0.5, 0.25)',
 '(0, 0.25, 0.25, 0.5)',
 '(0, 0.25, 0, 0.75)',
 '(0, 0, 1, 0)',
 '(0, 0, 0.75, 0.25)',
 '(0, 0, 0

In [12]:
# Optimize the objective functions
# Return a list of scenario_class variable containing information about each scenario, factors and impacts

opt_resultats=functions.optimisation_one_obj(data,list_objective_functions_final,temporary_saving_file,open_file="n")

L'optimisation totale prend quelques secondes
opt. IWP Mid | Climate change, short term (kg CO2 eq (short)) has been optimized successfully
opt. IWP Mid | Fossil and nuclear energy use (MJ deprived) has been optimized successfully
opt. IWP End | Ecosystem quality | Ecosystem quality total red (PDF.m2.yr) has been optimized successfully
opt. IWP End | Human health | Human health total red (DALY) has been optimized successfully
opt. (1, 0, 0, 0) has been optimized successfully
opt. (0.75, 0.25, 0, 0) has been optimized successfully
opt. (0.75, 0, 0.25, 0) has been optimized successfully
opt. (0.75, 0, 0, 0.25) has been optimized successfully
opt. (0.5, 0.5, 0, 0) has been optimized successfully
opt. (0.5, 0.25, 0.25, 0) has been optimized successfully
opt. (0.5, 0.25, 0, 0.25) has been optimized successfully
opt. (0.5, 0, 0.5, 0) has been optimized successfully
opt. (0.5, 0, 0.25, 0.25) has been optimized successfully
opt. (0.5, 0, 0, 0.5) has been optimized successfully
opt. (0.25, 0.75

# Data storage and retrieving into pickle files

At each optimisation, you can store the calculated scenarios in a pikle file to then re use them

## Data storage

In [14]:
# Save the precedent results into pickle form

with open(pickle_file,"wb") as output_file :
    pickle.dump(opt_resultats, output_file)
